# Parallelization Lab

In this lab, you will be leveraging several concepts you have learned to obtain a list of links from a web page and crawl and index the pages referenced by those links - both sequentially and in parallel. Follow the steps below to complete the lab.

## Step 1: Use the requests library to retrieve the content from the URL below.

In [1]:
import requests

url = 'https://en.wikipedia.org/wiki/Data_science'

In [2]:
DS = requests.get(url)
DS

<Response [200]>

## Step 2: Use BeautifulSoup to extract a list of all the unique links on the page.

In [4]:
from bs4 import BeautifulSoup as bs

In [6]:
dscontenido = DS.content
sopads = bs(dscontenido, 'html.parser')


In [37]:
links = sopads.findAll({'a':'href'})
linkslista=[i.get('href') for i in links]
linkslista

[None,
 '#mw-head',
 '#searchInput',
 '/wiki/Information_science',
 '/wiki/Machine_learning',
 '/wiki/Data_mining',
 '/wiki/Statistical_classification',
 '/wiki/Cluster_analysis',
 '/wiki/Regression_analysis',
 '/wiki/Anomaly_detection',
 '/wiki/Automated_machine_learning',
 '/wiki/Association_rule_learning',
 '/wiki/Reinforcement_learning',
 '/wiki/Structured_prediction',
 '/wiki/Feature_engineering',
 '/wiki/Feature_learning',
 '/wiki/Online_machine_learning',
 '/wiki/Semi-supervised_learning',
 '/wiki/Unsupervised_learning',
 '/wiki/Learning_to_rank',
 '/wiki/Grammar_induction',
 '/wiki/Supervised_learning',
 '/wiki/Statistical_classification',
 '/wiki/Regression_analysis',
 '/wiki/Decision_tree_learning',
 '/wiki/Ensemble_learning',
 '/wiki/Bootstrap_aggregating',
 '/wiki/Boosting_(machine_learning)',
 '/wiki/Random_forest',
 '/wiki/K-nearest_neighbors_algorithm',
 '/wiki/Linear_regression',
 '/wiki/Naive_Bayes_classifier',
 '/wiki/Artificial_neural_network',
 '/wiki/Logistic_regre

## Step 3: Use list comprehensions with conditions to clean the link list.

There are two types of links, absolute and relative. Absolute links have the full URL and begin with http while relative links begin with a forward slash (/) and point to an internal page within the wikipedia.org domain. Clean the respective types of URLs as follows.

- Absolute Links: Create a list of these and remove any that contain a percentage sign (%).
- Relativel Links: Create a list of these, add the domain to the link so that you have the full URL, and remove any that contain a percentage sign (%).
- Combine the list of absolute and relative links and ensure there are no duplicates.

In [53]:
domain = 'http://wikipedia.org'
import re

In [68]:
absolute_links = []
for i in linkslista:
    j = str(i)
    if j.startswith('http'):
        absolute_links.append(j)

#todas las url tienen %

In [69]:
relative_links = []
for i in linkslista:
    j = str(i)
    if j.startswith('/'):
        relative_links.append(domain+j)
        


In [86]:
dos= set(absolute_links+relative_links)
combine_list = list(dos)

combine_list

['http://wikipedia.org/wiki/Reinforcement_learning',
 'http://wikipedia.org/wiki/Regression_analysis',
 'http://wikipedia.org/w/index.php?title=Data_science&printable=yes',
 'http://wikipedia.org/wiki/Data_mining',
 'https://doi.org/10.1007%2F978-4-431-65950-1_3',
 'http://wikipedia.org/wiki/List_of_datasets_for_machine-learning_research',
 'http://wikipedia.org/w/index.php?title=Data_science&action=edit&section=13',
 'http://wikipedia.org/wiki/Data_(computing)',
 'http://wikipedia.org/wiki/Data_reduction',
 'https://fa.wikipedia.org/wiki/%D8%B9%D9%84%D9%85_%D8%AF%D8%A7%D8%AF%D9%87%E2%80%8C%D9%87%D8%A7',
 'http://wikipedia.org/wiki/Decision_tree_learning',
 'http://wikipedia.org//www.worldcat.org/oclc/489990740',
 'http://wikipedia.org/wiki/Data_preservation',
 'http://wikipedia.org/wiki/Plotly',
 'http://wikipedia.org/wiki/Portal:Current_events',
 'http://wikipedia.org/w/index.php?title=Data_science&action=info',
 'http://wikipedia.org/wiki/Knowledge',
 'http://wikipedia.org/w/index.p

## Step 4: Use the os library to create a folder called *wikipedia* and make that the current working directory.

In [65]:
import os

path = '/Users/rube/datamex_082020/module-1/lab-parallelization/your-code/wikipedia'

In [66]:
os.mkdir(path)

## Step 5: Write a function called index_page that accepts a link and does the following.

- Tries to request the content of the page referenced by that link.
- Slugifies the filename using the `slugify` function from the [python-slugify](https://pypi.org/project/python-slugify/) library and adds a .html file extension.
    - If you don't already have the python-slugify library installed, you can pip install it as follows: `$ pip install python-slugify`.
    - To import the slugify function, you would do the following: `from slugify import slugify`.
    - You can then slugify a link as follows `slugify(link)`.
- Creates a file in the wikipedia folder using the slugified filename and writes the contents of the page to the file.
- If an exception occurs during the process above, just `pass`.

In [67]:
from slugify import slugify

In [91]:
def index_page(x):
    from slugify import slugify
    import requests
    from bs4 import BeautifulSoup as bs
    
    #leo
    try:
        resp = requests.get(x)
    except:
        pass
    
    sopa = bs(resp.content, 'html.parser')
    texto = sopa.text
    
    #creo archivos
    lugar = '/Users/rube/datamex_082020/module-1/lab-parallelization/your-code/wikipedia/'
    file = open(lugar+slugify(x)+'.html', "w")
    file.write(texto)
    file.close()
    
    

## Step 6: Sequentially loop through the list of links, running the index_page function each time.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [93]:
%%time
for i in combine_list:
    index_page(i)
    
    #fueron 6 minutos y 45 seg

CPU times: user 1min 3s, sys: 3.16 s, total: 1min 7s
Wall time: 6min 45s


## Step 7: Perform the page indexing in parallel and note the difference in performance.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [94]:
import multiprocessing as mp
import numpy as np

In [95]:
mp.cpu_count()

4

In [96]:
mp.Condition()

<Condition(<RLock(unknown, unknown)>, unknown)>

In [98]:
pool = mp.Pool(mp.cpu_count())
pool

In [99]:
%%time
res = pool.map(index_page, combine_list)
pool.close()
res
#fue un minuto y medio, en este ejercicio si funcionó el multiproceso

CPU times: user 139 ms, sys: 50 ms, total: 189 ms
Wall time: 1min 27s


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,